In [1]:
from tensorflow import keras
from keras import models
from keras import layers
from keras import optimizers, losses, metrics
from keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

# 데이터 로드

디코더 입력에 START가 들어가면 디코딩의 시작 의미. 반대로 디코더 출력에 END가 나오면 디코딩 종료

In [2]:
# 태그 단어
PAD = "<PADDING>"
STA = "<STA>"
END = "<END>"
OOV = "<OOV>"

# 태그 인덱스
PAD_INDEX = 0
STA_INDEX = 1
END_INDEX = 2
OOV_INDEX = 3

# 데이터 타입
ENCODER_INPUT = 0
DECODER_INPUT  = 1
DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수
max_sequences = 30

# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든 레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("[.,!?\"':;~()]")

# 챗봇 데이터 로드
chatbot_data = pd.read_csv('./ChatbotData.csv',encoding='utf-8')
question, answer = list(chatbot_data['Q']), list(chatbot_data['A'])

In [3]:
len(question)

11823

In [4]:
# 데이터의 일부만 학습에 사용
question = question[:100]
answer = answer[:100]

# 챗봇 데이터 출력
for i in range(10):
    print('Q:'+question[i])
    print('A:'+answer[i])
    print()

Q:12시 땡!
A:하루가 또 가네요.

Q:1지망 학교 떨어졌어
A:위로해 드립니다.

Q:3박4일 놀러가고 싶다
A:여행은 언제나 좋죠.

Q:3박4일 정도 놀러가고 싶다
A:여행은 언제나 좋죠.

Q:PPL 심하네
A:눈살이 찌푸려지죠.

Q:SD카드 망가졌어
A:다시 새로 사는 게 마음 편해요.

Q:SD카드 안돼
A:다시 새로 사는 게 마음 편해요.

Q:SNS 맞팔 왜 안하지ㅠㅠ
A:잘 모르고 있을 수도 있어요.

Q:SNS 시간낭비인 거 아는데 매일 하는 중
A:시간을 정하고 해보세요.

Q:SNS 시간낭비인데 자꾸 보게됨
A:시간을 정하고 해보세요.



# 단어 사전 생성

In [5]:
# 형태소 분석 함수 
def pos_tag(sentences):
    # KoNLPy 형태소 분석기 설정
    tagger = Okt()
    # 문장 품사 변수 초기화
    sentences_pos = []
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER,"",sentence)

        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        # 형태소 단위로 문자열을 끊고 싶다면, .morphs()를 사용하면 된다
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
    return sentences_pos

In [6]:
# 형태소 분석 수행
question = pos_tag(question)
answer = pos_tag(answer)
print(question)
print(answer)
# 형태소 분석으로 변환된 챗봇 데이터 출력
for i in range(10):
    print('Q:' + question[i])
    print('A:' + answer[i])
    print()

['12시 땡', '1 지망 학교 떨어졌어', '3 박 4일 놀러 가고 싶다', '3 박 4일 정도 놀러 가고 싶다', 'PPL 심하네', 'SD 카드 망가졌어', 'SD 카드 안 돼', 'SNS 맞팔 왜 안 하지 ㅠㅠ', 'SNS 시간 낭비 인 거 아는데 매일 하는 중', 'SNS 시간 낭비 인데 자꾸 보게 됨', 'SNS 보면 나 만 빼고 다 행복 해보여', '가끔 궁금해', '가끔 뭐 하는지 궁금해', '가끔 은 혼자 인게 좋다', '가난한 자의 설움', '가만 있어도 땀 난다', '가상 화폐 쫄딱 망함', '가스 불 켜고 나갔어', '가스 불 켜놓고 나온거 같아', '가스 비 너무 많이 나왔다', '가스 비 비싼데 감기 걸리겠어', '가스 비 장난 아님', '가장 확실한 건 뭘 까', '가족 여행 가기 로 했어', '가족 여행 고고', '가족 여행 어디 로 가지', '가족 있어', '가족 관계 알려 줘', '가족 끼리 여행 간다', '가족 들 보고 싶어', '가족 들 이랑 서먹해', '가족 들 이랑 서먹해졌어', '가족 들 이랑 어디 가지', '가족 들 이랑 여행 갈거야', '가족 여행 가야 지', '가족 이 누구 야', '가족 이랑 여행 가려고', '가족 한테 스트레스 풀었어', '가출 할까', '가출 해도 갈 데 가 없어', '간만 에 떨리니까 좋더라', '간만 에 쇼핑 중', '간만 에 휴식 중', '간식 뭐 먹을까', '간식 추천', '간장 치킨 시켜야지', '간접흡연 싫어', '갈까 말까 고민 돼', '갈까 말까', '감 말랭이 먹고 싶다', '감 말랭이 먹어야지', '감기 같 애', '감기 걸린 것 같아', '감기 기운 이 있어', '감기 들 거 같 애', '감기 가 오려나', '감기 약 이 없어', '감기 인거 같 애', '감미로운 목소리 좋아', '감정 이 쓰레기통 처럼 엉망 진창 이야', '감정 컨트롤 을 못 하겠어', '감정 컨트롤 이 안 돼', '감히 나를 무시 하는 애가 있어', '갑자기 나쁜 생각 이 막 들더라', '갑자기 눈물 

In [7]:
# 질문과 대답 문장들을 하나로 합침
sentences=[]
sentences.extend(question)
sentences.extend(answer)

words=[]

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)
# 길이가 0인 단어는 삭제
words = [word for word in words if len(word)>0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, STA, END, OOV]

질문과 대답 문장들을 합쳐서 전체 단어사전 만들기. 자연어 처리에서는 항상 이렇게 단어를 인덱스에 따라서 정리.그래야지 문장을 인덱스 배열로 바꿔서 임베딩 레이어에 넣을 수 있습니다. 
또한 모델의 출력에서 나온 인덱스를 다시 단어로 변환하는데도 필요

In [8]:
# 단어 개수
len(words)

454

In [9]:
# 단어 출력
words

['<PADDING>',
 '<STA>',
 '<END>',
 '<OOV>',
 '하는지',
 '질질',
 '일도',
 '부족했나',
 '연락',
 '나를',
 '하루',
 '간접흡연',
 '컨트롤',
 '시켜야지',
 '놓고',
 '비',
 '보고',
 '알아차려도',
 '갑자기',
 '드세요',
 '갈까',
 '싫다',
 '나온거',
 '떨리는',
 '수도',
 '있을까',
 '시간',
 '하세요',
 '가',
 '에요',
 '가려고',
 '가스',
 '즐거운',
 '좋아요',
 '힘들죠',
 '야',
 '낭비',
 '잊고',
 '중요해요',
 '않을',
 '수',
 '피',
 '물어봐서',
 '짧죠',
 '패턴',
 '놀러',
 '수영장',
 '가족',
 '강원도',
 '나오세요',
 '감',
 '너무',
 '마음',
 '게',
 '기회',
 '건',
 '중',
 '궁금해',
 '가야',
 'ㅠㅠ',
 '습관',
 '씨방',
 '까지',
 '힘든데',
 '이럴',
 '중요한',
 '반',
 '누굴',
 '세수',
 '편해요',
 '주는',
 '소중해요',
 '처럼',
 '바빠서',
 '절약',
 '했어',
 '들더라',
 '취미',
 '애',
 '막',
 '만나지',
 '해도',
 '사세요',
 '옆',
 '가까워질',
 '연인',
 '입어',
 '단',
 '곳',
 '자랑',
 '역시',
 '룩',
 '3',
 '확실한',
 '일',
 '간식',
 '싶다',
 '내',
 '생활',
 '낭비하지',
 '은',
 '에는',
 '모두',
 '인거',
 '드는',
 '득템',
 '당황',
 '목소리',
 '보면',
 '먹었는데',
 '끌',
 '키울',
 '쉬는',
 '못',
 '자꾸',
 '업무',
 '드립니다',
 '제일',
 '간다',
 '사는게',
 '자의',
 '켜고',
 '왔나',
 '데',
 '좋겠다',
 '스트레스',
 '서로',
 '입어볼까',
 '물어',
 '무시',
 '부모님',
 '감미로운',
 '병원',
 '개념',
 '가난한',
 '불

In [12]:
# 단어와 인덱스의 딕셔너리 생성
word_to_index = {word: index for index, word in enumerate(words)}
index_to_word = {index: word for index, word in enumerate(words)}

In [13]:
# 단어->인덱스
# 문장을 인덱스로 변환하여 모델 입력으로 사용
word_to_index

{'<PADDING>': 0,
 '<STA>': 1,
 '<END>': 2,
 '<OOV>': 3,
 '하는지': 4,
 '질질': 5,
 '일도': 6,
 '부족했나': 7,
 '연락': 8,
 '나를': 9,
 '하루': 10,
 '간접흡연': 11,
 '컨트롤': 12,
 '시켜야지': 13,
 '놓고': 14,
 '비': 15,
 '보고': 16,
 '알아차려도': 17,
 '갑자기': 18,
 '드세요': 19,
 '갈까': 20,
 '싫다': 21,
 '나온거': 22,
 '떨리는': 23,
 '수도': 24,
 '있을까': 25,
 '시간': 26,
 '하세요': 27,
 '가': 28,
 '에요': 29,
 '가려고': 30,
 '가스': 31,
 '즐거운': 32,
 '좋아요': 33,
 '힘들죠': 34,
 '야': 35,
 '낭비': 36,
 '잊고': 37,
 '중요해요': 38,
 '않을': 39,
 '수': 40,
 '피': 41,
 '물어봐서': 42,
 '짧죠': 43,
 '패턴': 44,
 '놀러': 45,
 '수영장': 46,
 '가족': 47,
 '강원도': 48,
 '나오세요': 49,
 '감': 50,
 '너무': 51,
 '마음': 52,
 '게': 53,
 '기회': 54,
 '건': 55,
 '중': 56,
 '궁금해': 57,
 '가야': 58,
 'ㅠㅠ': 59,
 '습관': 60,
 '씨방': 61,
 '까지': 62,
 '힘든데': 63,
 '이럴': 64,
 '중요한': 65,
 '반': 66,
 '누굴': 67,
 '세수': 68,
 '편해요': 69,
 '주는': 70,
 '소중해요': 71,
 '처럼': 72,
 '바빠서': 73,
 '절약': 74,
 '했어': 75,
 '들더라': 76,
 '취미': 77,
 '애': 78,
 '막': 79,
 '만나지': 80,
 '해도': 81,
 '사세요': 82,
 '옆': 83,
 '가까워질': 84,
 '연인': 85,
 '입어': 86,
 '단': 87,

In [14]:
# 인덱스 -> 단어
# 문장을 인덱스로 변환하여 모델 입력으로 사용
index_to_word

{0: '<PADDING>',
 1: '<STA>',
 2: '<END>',
 3: '<OOV>',
 4: '하는지',
 5: '질질',
 6: '일도',
 7: '부족했나',
 8: '연락',
 9: '나를',
 10: '하루',
 11: '간접흡연',
 12: '컨트롤',
 13: '시켜야지',
 14: '놓고',
 15: '비',
 16: '보고',
 17: '알아차려도',
 18: '갑자기',
 19: '드세요',
 20: '갈까',
 21: '싫다',
 22: '나온거',
 23: '떨리는',
 24: '수도',
 25: '있을까',
 26: '시간',
 27: '하세요',
 28: '가',
 29: '에요',
 30: '가려고',
 31: '가스',
 32: '즐거운',
 33: '좋아요',
 34: '힘들죠',
 35: '야',
 36: '낭비',
 37: '잊고',
 38: '중요해요',
 39: '않을',
 40: '수',
 41: '피',
 42: '물어봐서',
 43: '짧죠',
 44: '패턴',
 45: '놀러',
 46: '수영장',
 47: '가족',
 48: '강원도',
 49: '나오세요',
 50: '감',
 51: '너무',
 52: '마음',
 53: '게',
 54: '기회',
 55: '건',
 56: '중',
 57: '궁금해',
 58: '가야',
 59: 'ㅠㅠ',
 60: '습관',
 61: '씨방',
 62: '까지',
 63: '힘든데',
 64: '이럴',
 65: '중요한',
 66: '반',
 67: '누굴',
 68: '세수',
 69: '편해요',
 70: '주는',
 71: '소중해요',
 72: '처럼',
 73: '바빠서',
 74: '절약',
 75: '했어',
 76: '들더라',
 77: '취미',
 78: '애',
 79: '막',
 80: '만나지',
 81: '해도',
 82: '사세요',
 83: '옆',
 84: '가까워질',
 85: '연인',
 86: '입어',
 87: '단',

# 전처리

In [15]:
# 문장을 인덱스로 변환
def convert_text_to_index(sentences, vocabulary, type):
    sentences_index = []

    # 모든 문장에 대해서 반복
    for sentence in sentences:
        sentence_index = []

        # 디코더 입력일 경우 맨 앞에 START태그 추가
        if type==DECODER_INPUT:
            sentence_index.extend([vocabulary[STA]])

        # 문장의 단어들을 띄어쓰기로 분리
        for word in sentence.split():
            if vocabulary.get(word) is not None:
                #사전에 있는 단어면 해당 인덱스를 추가
                sentence_index.extend([vocabulary[word]])
            else:
                #사전에 없는 단어면 OOV인덱스 추가
                sentence_index.extend([vocabulary[OOV]])
        # 최대 길이 검사
        if type == DECODER_TARGET:
            # 디코더 목표일 경우 맨 뒤에 END태그 추가
            if len(sentence_index) >= max_sequences:
                sentence_index = sentence_index[:max_sequences-1] + [vocabulary[END]]
            else:
                sentence_index+=[vocabulary[END]]
        else:
            if len(sentence_index) > max_sequences:
                sentence_index = sentence_index[:max_sequences]
        
        # 최대 길이에 없는 공간은 패딩 인덱스로 채움
        sentence_index+=(max_sequences-len(sentence_index))*[vocabulary[PAD]]

        # 문장의 인덱스 배열을 추가
        sentences_index.append(sentence_index)

    return np.asarray(sentences_index)
    

원래 seq2seq는 디코더의 현재 출력이 디코더의 다음 입력으로 들어갑니다. 다만 학습에서는 굳이 이렇게 하지 않고 디코더 입력과 디코더 출력의 데이터를 각각 만듭니다.

그러나 예측시에는 이런 방식이 불가능. 출력값을 미리 알지 못하기 때문에, 디코더 입력을 사전에 생성할 수가 없습니다. 이런 문제를 해결하기 위해 훈련 모델과 예측 모델을 따로 구성해야 합니다.

In [16]:
# 인코더 입력 인덱스 변환
x_encoder = convert_text_to_index(question, word_to_index, ENCODER_INPUT)

# 첫번째 인코더 입력 출력(12시 땡)
x_encoder[0]

array([386, 202,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [17]:
# 디코더 입력 인덱스 변환
x_decoder = convert_text_to_index(answer, word_to_index, DECODER_INPUT)

# 첫번째 디코더 입력 출력(START 하루 가 또 가네요)
x_decoder[0]

array([  1,  10,  28, 413, 401,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0])

In [18]:
# 디코더 목표 인덱스 변환
y_decoder = convert_text_to_index(answer,word_to_index, DECODER_TARGET)

# 첫 번재 디코더 목표 출력(하루 가 또 가네요 END)
y_decoder[0] # 2가 end tag
len(y_decoder)

100

In [19]:
# 원핫인코딩 초기화
one_hot_data = np.zeros((len(y_decoder), max_sequences, len(words))) #(100,30,454)===>(batch, 문장 길이, 단어개수)

# 디코더 목표를 원핫인코딩으로 변환 #DECODER_TARGET
# 학습 시 입력은 인덱스이지만, 출력은 원핫인코딩 형식
for i, sequence in enumerate(y_decoder):
    for j, index in enumerate(sequence):
        one_hot_data[i,j,index]=1

# 디코더 목표 설정
y_decoder = one_hot_data

# 첫번째 디코더 목표 출력
y_decoder[0]
print(y_decoder[0].shape) #(30,454)
print(y_decoder.shape)    #(100,30,454) (30,454)가 총 100개 있음

(30, 454)
(100, 30, 454)


인코더 입력과 디코더 입력은 임베딩 레이어에 들어가는 인덱스 배열입니다. 반면에 디코더 출력은 원핫인코딩 형식이어야 합니다. 디코더의 마지막 Dense레이어에서 Softmax로 나오기 때문

# 모델 생성

In [23]:
#-----------------------------------
# 훈련 모델 인코더 정의
#-----------------------------------

# 입력 문장의 인덱스 시퀀스를 입력으로 받음
encoder_inputs = layers.Input(shape=(None,),name='A1')    

# 임베딩 레이어
encoder_outputs = layers.Embedding(len(words), embedding_dim,name='A2')(encoder_inputs) #(454,100) #[(None,None,100)]

# return_state가 True면 상태값 리턴
# LSTM은 state_h와 state_c 2개의 상태 존재
# recurrent_dropout은 state 삭제시키는 것
encoder_outputs, state_h, state_c = layers.LSTM(lstm_hidden_dim, dropout=0.1, recurrent_dropout=0.5, return_state=True,name='A5')(encoder_outputs) #[(None,128),None// parameters: 117,248(4*128((128+1(bias)+100(embedding_dim)))

# 히든 상태와 셀 상태를 하나로 묶음
encoder_states = [state_h, state_c]

#-------------------------------------
# 훈련 모델 디코더 정의
#------------------------------------

# 목표 문장의 인덱스 시퀀스를 입력으로 받음
decoder_inputs = layers.Input(shape=(None,),name='A3')

# 임베딩 레이어
decoder_embedding = layers.Embedding(len(words),embedding_dim,name='A4') #(454,100)
decoder_outputs = decoder_embedding(decoder_inputs)
print(decoder_outputs)

# 인코더와 달리 return_sequences를 True로 설정하여 모든 타임스텝 출력값 리턴
# 모든 타임 스텝의 출력값들을 다음 레이어의 Dense()로 처리가히 위함
decoder_lstm  = layers.LSTM(lstm_hidden_dim, dropout=0.1, recurrent_dropout=0.5, return_state=True, return_sequences=True,name='A6')


# initial_state를 인코더의 상태로 초기화
decoder_outputs,_,_=decoder_lstm(decoder_outputs, initial_state=encoder_states)

# 단어의 개수만큼 노드의 개수를 설정하여 원핫 형식으로 각 단어 인덱스를 출력
decoder_dense = layers.Dense(len(words),activation='softmax',name='A7')
decoder_outputs = decoder_dense(decoder_outputs)

#-------------------------------------------
# 훈련 모델 정의
#-------------------------------------------

# 입력과 출력으로 함수형 API모델 생성
model = models.Model([encoder_inputs, decoder_inputs], decoder_outputs)

# 학습 방법 설정
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])
model.summary()


Tensor("A4/embedding_lookup_2/Identity_1:0", shape=(None, None, 100), dtype=float32)
Model: "functional_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
A1 (InputLayer)                 [(None, None)]       0                                            
__________________________________________________________________________________________________
A3 (InputLayer)                 [(None, None)]       0                                            
__________________________________________________________________________________________________
A2 (Embedding)                  (None, None, 100)    45400       A1[0][0]                         
__________________________________________________________________________________________________
A4 (Embedding)                  (None, None, 100)    45400       A3[0][0]                         
__

지금까지의 예제는 Sequential방식의 모델이었습니다. 이번에는 함수형 API모델 사용. 인코더와 디코더가 따로 분리되어야 하는데, 단순히 레이어를 추가하여 붙이는 순차형으로는 구현이 불가능

Model()함수로 입력과 출력을 따로 설정하며 모델 만듭니다. 그 다음 compile과 fit은 이전과 동일하게 적용하시면 됩니다. 

In [25]:
#-----------------------------------
#예측 모델 인코더 정의
#-----------------------------------

# 훈련 모델의 인코더 상태를 사용하여 예측 모델 인코더 설정
# encoder_states = [state_h, state_c]
encoder_model = models.Model(inputs=encoder_inputs, outputs=encoder_states) # hidden상태값이 output

#-----------------------------------
# 예측 모델 디코더 정의
#-----------------------------------

# 예측시에는 훈련시와 달리 타임 스텝을 한 단계씩 수행
# 매번 이전 디코더 상태를 입력으로 받아서 새로 설정
decoder_state_input_h = layers.Input(shape=(lstm_hidden_dim,),name='B1')
decoder_state_input_c = layers.Input(shape=(lstm_hidden_dim,),name='B2')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 임베딩 레이어
decoder_outputs = decoder_embedding(decoder_inputs) # A4

# LSTM레이어
decoder_outputs, state_h, state_c = decoder_lstm(decoder_outputs, initial_state = decoder_states_inputs) # A6

# 히든 상태와 셀 상태를 하나로 묶음
decoder_states = [state_h, state_c]

# Dense레이어를 통해 원핫 형식으로 각 단어 인덱스를 출력
decoder_outputs = decoder_dense(decoder_outputs)

# 예측 모델 디코더 설정
decoder_model = models.Model([decoder_inputs]+decoder_states_inputs, [decoder_outputs]+decoder_states)
decoder_model.summary()


Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
A3 (InputLayer)                 [(None, None)]       0                                            
__________________________________________________________________________________________________
A4 (Embedding)                  (None, None, 100)    45400       A3[0][0]                         
__________________________________________________________________________________________________
B1 (InputLayer)                 [(None, 128)]        0                                            
__________________________________________________________________________________________________
B2 (InputLayer)                 [(None, 128)]        0                                            
_______________________________________________________________________________________

예측 모델은 이미 학습된 훈련 모델의 레이어들을 그대로 재사용. 예측 모델 인코더는 훈련 모델 인코더와 동일. 그러나 예측 모델 디코더는 매번 LSTM상태값을 입력 받음. 또한 디코더의 LSTM상태를 출력값과 같이 내보내서, 다음 번 입력에 넣습니다.

이렇게 하는 이유는 LSTM을 딱 한번의 타임스텝만 실행하기 때문. 그래서 매번 상태값을 새로 초기화 해야 합니다. 이와 반대로 훈련할 때는 문장 전체를 계속 LSTM으로 돌리기 때문에 자동으로 상태값이 전달됩니다. 

# 훈련 및 테스트

In [26]:
# 인덱스를 문장으롭 변환
def convert_index_to_text(indexs, vocabulary):
    sentence=''

    #모든 문장에 대해서 반복
    for index in indexs:
        if index == END_INDEX:
            #종료 인덱스면 중지
            break
        elif vocabulary.get(index) is not None:
            # 사전에 있는 인덱스면 해당 단어를 추가
            sentence += vocabulary[index]
        else:
            sentence+=vocabulary[OOV_INDEX]
        # 빈칸 추가
        sentence += ' '
    return sentence

In [27]:
# epoch 반복
for epoch in range(20):
    print('Total Epoch:', epoch+1)

    # 훈련 시작
    history = model.fit([x_encoder,x_decoder], # input
                        y_decoder,  # output
                        epochs=100,
                        batch_size=64,
                        verbose=0)
    
    # 정확도와 손실 출력
    print('accuracy:',history.history['acc'][-1])
    print('loss:',history.history['loss'][-1])

    # 문장 예측 테스트
    # (3박 4일 놀러 가고 싶다) -> (여행 은 언제나 좋죠)
    input_encoder = x_encoder[0].reshape(1, x_encoder[2].shape[0]) # (1,30)
    input_decoder = x_decoder[0].reshape(1, x_decoder[2].shape[0])
    results = model.predict([input_encoder, input_decoder])

    # 결과의 원핫인코딩 형식을 인덱스로 변환
    # 1축을 기준으로 가장 높은 값의 위치를 구함
    indexs = np.argmax(results[0],1)

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)
    print(sentence)
    print()

Total Epoch: 1
accuracy: 0.8373333215713501
loss: 0.7063806056976318
저 이 이 

Total Epoch: 2
accuracy: 0.9309999942779541
loss: 0.3437199890613556
벗어나는 도 정 가네요 

Total Epoch: 3
accuracy: 0.9673333168029785
loss: 0.14309640228748322
벗어나는 가 또 가네요 

Total Epoch: 4
accuracy: 0.9733333587646484
loss: 0.08436500281095505
벗어나는 가 또 가네요 

Total Epoch: 5
accuracy: 0.9789999723434448
loss: 0.06543173640966415
벗어나는 가 또 가네요 

Total Epoch: 6
accuracy: 0.9836666584014893
loss: 0.04921303689479828
벗어나는 가 또 가네요 

Total Epoch: 7
accuracy: 0.9866666793823242
loss: 0.035948511213064194
낮잠 가 또 가네요 

Total Epoch: 8
accuracy: 0.9940000176429749
loss: 0.018876325339078903
콕 가 또 가네요 

Total Epoch: 9
accuracy: 0.9953333139419556
loss: 0.013263976201415062
하루 가 또 가네요 

Total Epoch: 10
accuracy: 0.9980000257492065
loss: 0.007462525740265846
하루 가 또 가네요 

Total Epoch: 11
accuracy: 0.9973333477973938
loss: 0.007120341062545776
낮잠 가 또 가네요 

Total Epoch: 12
accuracy: 0.9973333477973938
loss: 0.005074048414826393
낮잠 가 또

In [34]:
x_encoder[2].shape

(30,)

In [35]:
input_encoder = x_encoder[2].reshape(1,30)
input_encoder.shape

(1, 30)

In [36]:
results[0]
print(results[0].shape) #(30,454)
print(results.shape)    #(1,30,454)

(30, 454)
(1, 30, 454)


학습이 진행될수록 예측 문장이 제대로 생성되는 것을 볼 수 있다. 다만 여기서의 예측은 단순히 테스트를 위한 것이라, 인코더 입력과 디코더 입력 데이터가 동시에 사용. 아래 문장 생성에서는 예측 모델을 적용하기 때문에, 오직 인코더 입력 데이터만 집어 넣습니다. 

In [37]:
# 모델 저장
encoder_model.save('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model.save('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 저장
with open('./model/word_to_index.pkl','wb')as f:
    pickle.dump(word_to_index, f, pickle.HIGHEST_PROTOCOL)
with open('./model/index_to_word.pkl','wb')as f:
    pickle.dump(index_to_word,f,pickle.HIGHEST_PROTOCOL)


<pickle모듈><br>
일반 텍스트를 파일로 저장할 떄는 파일 입출력 이용
하지만 리스트나 클래스같은 텍스트가 아닌 자료형은 일반적인 파일 입출력 방법으로는 데이터를 저장하거나 불러올 수 없다. <br>
pickle모듈을 이용하면 원하는 데이터를 자료형의 변경 없이 파일로 저장하여 그대로 로드할 수 있다. <br>
pickle로 데이터를 저장하거나 불러올때는 파일을 바이트형식으로 읽거나 써야한다.(wb,rb)

pickle.load()는 한줄씩 데이터를 읽어오고
pickle.dump()는 뭉탱이로 읽어옴


# 문장 생성

In [38]:
# 모델 파일 로드
encoder_model = models.load_model('./model/seq2seq_chatbot_encoder_model.h5')
decoder_model = models.load_model('./model/seq2seq_chatbot_decoder_model.h5')

# 인덱스 파일 로드
with open('./model/word_to_index.pkl','rb') as f:
    word_to_index = pickle.load(f)
with open('./model/index_to_word.pkl','rb') as f:
    index_to_word  = pickle.load(f)

In [39]:
# 예측을 위한 입력 생성
def make_predict_input(sentence):

    sentences = []
    sentences.append(sentence)
    sentences = pos_tag(sentences) # 형태소 분석
    input_seq = convert_text_to_index(sentences, word_to_index, ENCODER_INPUT) # 인덱스화

    return input_seq


In [40]:
# 텍스트 생성
def generate_text(input_seq):

    # 입력을 인코더에 넣어 마지막 상태 구함
    states = encoder_model.predict(input_seq) # 인덱스

    # 목표 시퀀스 초기화
    # 문장 1개, 토큰 1개
    target_seq = np.zeros((1,1))

    # 목표 시퀀스의 첫 번째에 <START> 태그 추가
    target_seq[0,0] = STA_INDEX

    # 인덱스 초기화
    indexs = []

    # 디코더 타임 스탭 반복
    while 1:
        # 디코더로 현재 타임 스텝 출력 구함
        # 처음에는 인코더 상태를, 다음부터 이전 디코더 상태로 초기화
        decoder_outputs, state_h, state_c = decoder_model.predict([target_seq]+states) # start + encoder_state(문맥벡터)

        # 결과의 원핫인코딩 형식을 인덱스로 변환
        index = np.argmax(decoder_outputs[0,0,:])
        indexs.append(index)

        # 종료 검사
        if index == END_INDEX or len(indexs) >= max_sequences:
            break
        # 목표 시퀀스를 바로 이전의 출력으로 설정
        target_seq = np.zeros((1,1))
        target_seq[0,0] = index     # 다음 출력할 때 input_index

        # 디코더의 이전 상태를 다음 디코더 예측에 사용
        states = [state_h, state_c] # 다음 출력할 때 input_state

    # 인덱스를 문장으로 변환
    sentence = convert_index_to_text(indexs, index_to_word)

    return sentence

제일 첫 단어는 START로 시작. 그리고 출력으로 나온 인덱스를 디코더 입력으로 넣고 다시 예측 반복. 상태값을 받아 다시 입력으로 같이 넣는 것에 주의. END태그가 나오면 문장 생성 종료

In [41]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('친구랑 싸웠어요')
input_seq

array([[294, 319,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [42]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'그 사람 도 그럴 거 예요 '

In [50]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('중요한 친구랑 심하게 싸웠어요')
input_seq

array([[ 65, 294, 319,   3,   3,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [51]:
# 예측 모델로 텍스트 생성
sentence = generate_text(input_seq)
sentence

'당신 이 요 '

데이터셋 문장에서는 없던 '같이'를 단어를 추가해 보았습니다. 그래도 비슷한 의미란 것을 파악하여 동일한 답변이 나왔습니다.

In [52]:
# 문장을 인덱스로 변환
input_seq = make_predict_input('친구 선물을 뭐로 하면 좋을까요')
input_seq

array([[294,   3, 136, 333, 195, 388,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0]])

In [53]:
sentence = generate_text(input_seq)
sentence

'당신 의 운 을 믿어 보세요 '